# Imports

In [19]:
import pandas as pd

# Find best Learning Rate

In [20]:
all_lrs = pd.DataFrame(columns=['lr', 'SD MAE (clipcount)', 'Ours MAE (clipcount)', 'SD MAE (YOLO)', 'Ours MAE (YOLO)', 'SD CLIP', 'Ours CLIP'])
lrs = [0.00025, 0.00125, 0.01, 0.05, 0.1]

for lr in lrs:
    curr = pd.read_pickle(f"experiments/lr/experiment_{str(lr)[2:]}lr.pkl")
    curr = curr[curr['is_clipcount'] == True]
    all_lrs = pd.concat([all_lrs, pd.DataFrame
    ([
        {
            "lr" : lr,
            'SD MAE (clipcount)' : curr['sd_count_diff'].mean(),
            'Ours MAE (clipcount)' : curr['sd_optimized_count_diff'].mean(),
            'SD MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_count_diff2'].mean(),
            'Ours MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_optimized_count_diff2'].mean(),
            'SD CLIP' : 1-curr['actual_relevance_score'].mean(),
            'Ours CLIP' : 1-curr['optimized_relevance_score'].mean()
        }
    ])], ignore_index=True)
    
all_lrs

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_18448\3185428557.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_lrs = pd.concat([all_lrs, pd.DataFrame


,lr,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,0.00025,21.358131,16.590973,15.282051,15.102564,0.705159,0.704340
1,0.00125,21.681614,10.087102,15.282051,13.230769,0.706907,0.705577
2,0.01000,22.110869,6.664971,15.282051,10.743590,0.704704,0.704778
3,0.05000,22.070056,5.242243,15.282051,12.615385,0.704478,0.703470
4,0.10000,21.524450,5.171116,15.282051,11.589744,0.704102,0.703690


In [21]:
best_lr_index = all_lrs["Ours MAE (YOLO)"].idxmin()
best_lr = lrs[best_lr_index]
best_lr

0.01

In [22]:
best_lr_df = pd.read_pickle(f"experiments/lr/experiment_{str(best_lr)[2:]}lr.pkl")
best_lr_df = best_lr_df[best_lr_df['is_clipcount'] == True]

# SD (clipcount) MAE
sd_clipcount_mae = best_lr_df.groupby('amount').agg({'sd_count_diff': 'mean'}).reset_index()
sd_clipcount_mae.rename(columns={'sd_count_diff': 'SD MAE (clipcount)'}, inplace=True)

# Ours (clipcount) MAE
ours_clipcount_mae = best_lr_df.groupby('amount').agg({'sd_optimized_count_diff': 'mean'}).reset_index()
ours_clipcount_mae.rename(columns={'sd_optimized_count_diff': 'Ours MAE (clipcount)'}, inplace=True)

# SD (yolo) MAE
sd_yolo_mae = best_lr_df[best_lr_df['is_yolo'] == True].groupby('amount').agg({'sd_count_diff2': 'mean'}).reset_index()
sd_yolo_mae.rename(columns={'sd_count_diff2': 'SD MAE (YOLO)'}, inplace=True)

# Ours (yolo) MAE
ours_yolo_mae = best_lr_df[best_lr_df['is_yolo'] == True].groupby('amount').agg({'sd_optimized_count_diff2': 'mean'}).reset_index()
ours_yolo_mae.rename(columns={'sd_optimized_count_diff2': 'Ours MAE (YOLO)'}, inplace=True)

# SD (CLIP)
sd_clip = (1 - best_lr_df.groupby('amount').agg({'actual_relevance_score': 'mean'})).reset_index()
sd_clip.rename(columns={'actual_relevance_score': 'SD CLIP'}, inplace=True)

# Ours (CLIP)
ours_clip = (1 - best_lr_df.groupby('amount').agg({'optimized_relevance_score': 'mean'})).reset_index()
ours_clip.rename(columns={'optimized_relevance_score': 'Ours CLIP'}, inplace=True)

# Merge all dataframes
merged_df = sd_clipcount_mae.merge(ours_clipcount_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(ours_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_clip, on='amount', how='outer')
merged_df = merged_df.merge(ours_clip, on='amount', how='outer')

merged_df

,amount,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,5,11.134555,4.616883,4.384615,4.153846,0.701925,0.700651
1,15,23.757216,5.755914,18.538462,13.076923,0.706844,0.705809
2,25,31.544502,9.654973,22.923077,15.0,0.705349,0.707909


# Find best lambda

In [23]:
all_lambdas = pd.DataFrame(columns=['lambda', 'SD MAE (clipcount)', 'Ours MAE (clipcount)', 'SD MAE (YOLO)', 'Ours MAE (YOLO)', 'SD CLIP', 'Ours CLIP'])
lambdas = [0,5,25,100]

for _lambda in lambdas:
    curr = pd.read_pickle(f"experiments/lambda/experiment_{_lambda}lambda{str(best_lr)[2:]}lr.pkl")
    curr = curr[curr['is_clipcount'] == True]
    all_lambdas = pd.concat([all_lambdas, pd.DataFrame
    ([
        {
            "lambda" : _lambda,
            'SD MAE (clipcount)' : curr['sd_count_diff'].mean(),
            'Ours MAE (clipcount)' : curr['sd_optimized_count_diff'].mean(),
            'SD MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_count_diff2'].mean(),
            'Ours MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_optimized_count_diff2'].mean(),
            'SD CLIP' : 1-curr['actual_relevance_score'].mean(),
            'Ours CLIP' : 1-curr['optimized_relevance_score'].mean()
        }
    ])], ignore_index=True)
    
all_lambdas

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_18448\2018212663.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_lambdas = pd.concat([all_lambdas, pd.DataFrame


,lambda,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,0,21.409582,6.091541,15.282051,12.589744,0.706393,0.705208
1,5,21.590128,5.584012,15.282051,11.230769,0.706846,0.705183
2,25,21.315139,5.704018,15.282051,11.871795,0.705355,0.704143
3,100,21.409582,5.990197,15.282051,12.282051,0.706393,0.703284


In [24]:
best_lambda_index = all_lambdas["Ours MAE (YOLO)"].idxmin()
best_lambda = lambdas[best_lambda_index]
best_lambda

5

In [25]:
best_lambda_df = pd.read_pickle(f"experiments/lambda/experiment_{best_lambda}lambda{str(best_lr)[2:]}lr.pkl")
best_lambda_df = best_lambda_df[best_lambda_df['is_clipcount'] == True]

# SD (clipcount) MAE
sd_clipcount_mae = best_lambda_df.groupby('amount').agg({'sd_count_diff': 'mean'}).reset_index()
sd_clipcount_mae.rename(columns={'sd_count_diff': 'SD MAE (clipcount)'}, inplace=True)

# Ours (clipcount) MAE
ours_clipcount_mae = best_lambda_df.groupby('amount').agg({'sd_optimized_count_diff': 'mean'}).reset_index()
ours_clipcount_mae.rename(columns={'sd_optimized_count_diff': 'Ours MAE (clipcount)'}, inplace=True)

# SD (yolo) MAE
sd_yolo_mae = best_lambda_df[best_lambda_df['is_yolo'] == True].groupby('amount').agg({'sd_count_diff2': 'mean'}).reset_index()
sd_yolo_mae.rename(columns={'sd_count_diff2': 'SD MAE (YOLO)'}, inplace=True)

# Ours (yolo) MAE
ours_yolo_mae = best_lambda_df[best_lambda_df['is_yolo'] == True].groupby('amount').agg({'sd_optimized_count_diff2': 'mean'}).reset_index()
ours_yolo_mae.rename(columns={'sd_optimized_count_diff2': 'Ours MAE (YOLO)'}, inplace=True)

# SD (CLIP)
sd_clip = (1 - best_lambda_df.groupby('amount').agg({'actual_relevance_score': 'mean'})).reset_index()
sd_clip.rename(columns={'actual_relevance_score': 'SD CLIP'}, inplace=True)

# Ours (CLIP)
ours_clip = (1 - best_lambda_df.groupby('amount').agg({'optimized_relevance_score': 'mean'})).reset_index()
ours_clip.rename(columns={'optimized_relevance_score': 'Ours CLIP'}, inplace=True)

# Merge all dataframes
merged_df = sd_clipcount_mae.merge(ours_clipcount_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(ours_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_clip, on='amount', how='outer')
merged_df = merged_df.merge(ours_clip, on='amount', how='outer')

merged_df

,amount,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,5,9.592247,4.918367,4.384615,3.384615,0.703157,0.700211
1,15,23.035197,5.362774,18.538462,16.923077,0.709599,0.705943
2,25,32.271633,6.481712,22.923077,13.384615,0.707795,0.709448


# Explore best lambda

### 1. Compare with ControlNet Baseline

In [26]:
# ControlNet
controlnet = pd.read_pickle(f"experiments/experiment_controlnet_1.pkl")
controlnet = controlnet[controlnet['is_clipcount'] == True]

# dynamic scale factor
dynamic_scale = pd.read_pickle(f"experiments/experiment_dynamic_scale_factor_01.pkl")
dynamic_scale = dynamic_scale[dynamic_scale['is_clipcount'] == True]
dynamic_scale = dynamic_scale[dynamic_scale['is_yolo'] == True]

sd_mae_clipcount = best_lambda_df['sd_count_diff'].mean()
controlnet_mae_clipcount = controlnet['sd_optimized_count_diff'].mean()
ours_mae_clipcount = best_lambda_df['sd_optimized_count_diff'].mean()
dynamic_scale_mae_clipcount = dynamic_scale['sd_optimized_count_diff'].mean()

sd_mae_yolo = best_lambda_df[best_lambda_df['is_yolo']==True]['sd_count_diff2'].mean()
controlnet_mae_yolo = controlnet[controlnet['is_yolo']==True]['sd_optimized_count_diff2'].mean()
ours_mae_yolo = best_lambda_df[best_lambda_df['is_yolo']==True]['sd_optimized_count_diff2'].mean()
dynamic_scale_mae_yolo = dynamic_scale[dynamic_scale['is_yolo']==True]['sd_optimized_count_diff2'].mean()

sd_clip = 1-best_lambda_df['actual_relevance_score'].mean()
controlnet_clip = 1-controlnet['optimized_relevance_score'].mean()
ours_mae_clip = 1-best_lambda_df['optimized_relevance_score'].mean()
dynamic_scale_mae_clip = 1-dynamic_scale['optimized_relevance_score'].mean()


data = {
    'MAE (clipcount)': [sd_mae_clipcount, controlnet_mae_clipcount, ours_mae_clipcount, dynamic_scale_mae_clipcount],
    'MAE (YOLO)': [sd_mae_yolo, controlnet_mae_yolo, ours_mae_yolo, dynamic_scale_mae_yolo],
    'CLIP': [sd_clip, controlnet_clip, ours_mae_clip, dynamic_scale_mae_clip]
}

pd.DataFrame(data, index=['SD', 'ControlNet', 'Ours', 'Ours (dynamic scale)'])

,MAE (clipcount),MAE (YOLO),CLIP
SD,21.590128,15.282051,0.706846
ControlNet,9.812914,15.000000,0.720257
Ours,5.584012,11.230769,0.705183
Ours (dynamic scale),15.888874,8.022222,0.701867


In [27]:
## Detailed Dynamic Scale Factor


# SD (clipcount) MAE
sd_clipcount_mae = dynamic_scale.groupby('amount').agg({'sd_count_diff': 'mean'}).reset_index()
sd_clipcount_mae.rename(columns={'sd_count_diff': 'SD MAE (clipcount)'}, inplace=True)

# Ours (clipcount) MAE
ours_clipcount_mae = dynamic_scale.groupby('amount').agg({'sd_optimized_count_diff': 'mean'}).reset_index()
ours_clipcount_mae.rename(columns={'sd_optimized_count_diff': 'Ours MAE (clipcount)'}, inplace=True)

# SD (yolo) MAE
sd_yolo_mae = dynamic_scale[dynamic_scale['is_yolo'] == True].groupby('amount').agg({'sd_count_diff2': 'mean'}).reset_index()
sd_yolo_mae.rename(columns={'sd_count_diff2': 'SD MAE (YOLO)'}, inplace=True)

# Ours (yolo) MAE
ours_yolo_mae = dynamic_scale[dynamic_scale['is_yolo'] == True].groupby('amount').agg({'sd_optimized_count_diff2': 'mean'}).reset_index()
ours_yolo_mae.rename(columns={'sd_optimized_count_diff2': 'Ours MAE (YOLO)'}, inplace=True)

# SD (CLIP)
sd_clip = (1 - dynamic_scale.groupby('amount').agg({'actual_relevance_score': 'mean'})).reset_index()
sd_clip.rename(columns={'actual_relevance_score': 'SD CLIP'}, inplace=True)

# Ours (CLIP)
ours_clip = (1 - dynamic_scale.groupby('amount').agg({'optimized_relevance_score': 'mean'})).reset_index()
ours_clip.rename(columns={'optimized_relevance_score': 'Ours CLIP'}, inplace=True)

# Merge all dataframes
merged_df = sd_clipcount_mae.merge(ours_clipcount_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(ours_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_clip, on='amount', how='outer')
merged_df = merged_df.merge(ours_clip, on='amount', how='outer')

merged_df

,amount,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,5,4.906011,16.002864,4.6,2.733333,0.696720,0.698453
1,15,23.152459,15.259892,18.733333,9.666667,0.703467,0.701213
2,25,35.251232,16.403866,22.2,11.666667,0.706853,0.705933


## Compare to standard SDXL with different number of diffusion steps

In [28]:
all_steps = pd.DataFrame(columns=['steps', 'SD (clipcount) MAE', 'Ours (clipcount) MAE', 'SD (yolo) MAE', 'Ours (yolo) MAE', 'SD (CLIP)', 'Ours (CLIP)'])

for step in range(1,5):
    curr = pd.read_pickle(f"experiments/token_reuse/experiment_{best_lambda}lambda{str(best_lr)[2:]}lr-eval-{step}.pkl")
    curr = curr[curr['is_clipcount'] == True]
    all_steps = pd.concat([all_steps, pd.DataFrame
    ([
        {
            "steps" : step,
            'SD (clipcount) MAE' : curr['sd_count_diff'].mean(),
            'Ours (clipcount) MAE' : curr['sd_optimized_count_diff'].mean(),
            'SD (yolo) MAE' : curr[curr['is_yolo'] == True]['sd_count_diff2'].mean(),
            'Ours (yolo) MAE' : curr[curr['is_yolo'] == True]['sd_optimized_count_diff2'].mean(),
            'SD (CLIP)' : 1-curr['actual_relevance_score'].mean(),
            'Ours (CLIP)' : 1-curr['optimized_relevance_score'].mean()
        }
    ])], ignore_index=True)
    
all_steps

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_18448\1421572942.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_steps = pd.concat([all_steps, pd.DataFrame


,steps,SD (clipcount) MAE,Ours (clipcount) MAE,SD (yolo) MAE,Ours (yolo) MAE,SD (CLIP),Ours (CLIP)
0,1,22.563755,5.596906,14.923077,11.230769,0.707426,0.705112
1,2,15.722917,6.610024,14.307692,12.615385,0.705697,0.705143
2,3,14.901099,7.136169,13.564103,12.205128,0.709214,0.707731
3,4,14.458994,7.712382,13.538462,11.461538,0.710098,0.708994
